### link to the Kaggle dataset named final.csv : https://www.kaggle.com/datasets/yelexa/spotify200

In [1]:
pip install kaggle

In [2]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
tor

In [3]:
import gradio as gr
import pandas as pd
import os
import kagglehub
import plotly.express as px


def get_df(link):
    dataset_path = kagglehub.dataset_download(link)
    csv_files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]
    if csv_files:
        csv_path = os.path.join(dataset_path, csv_files[0])
        df = pd.read_csv(csv_path)
    else:
        raise FileNotFoundError("No CSV file found in the dataset folder.")
    return df


def load_countries(link):
    df = get_df(link)
    countries = sorted(df['country'].dropna().unique().tolist())
    return df, gr.update(choices=countries, value=countries[0] if countries else None), None, None

def process_songs(df, country):
    df["full_track"] = df["artist_names"] + " - " + df["track_name"]
    songs = sorted(df[df['country'] == country]['full_track'].dropna().unique().tolist())
    return songs

def load_songs(df, country):
    if df is None or country is None:
        return [], None
    df["full_track"] = df["artist_names"] + " - " + df["track_name"]
    songs = sorted(df[df['country'] == country]['full_track'].dropna().unique().tolist())
    return gr.update(choices=songs, value=songs[0] if songs else None), None

def plot_song(df, country, song):
    if df is None or country is None or song is None:
        return None

    song_df = df[(df['country'] == country) & (df['full_track'] == song)]
    if song_df.empty:
        return None

    song_df['week'] = pd.to_datetime(song_df['week'])
    song_df = song_df.sort_values(by="week")

    fig = px.line(
        song_df,
        x='week',
        y='rank',
        title=f'{song} ranking over weeks in {country}',
        labels={'rank': 'Rank', 'week': 'Week'},
        markers=True
    )
    fig.update_yaxes(autorange="reversed",tickmode='linear', dtick=10)
    return fig

def load_dataset_and_countries(link):
        df, country_update, song_update, plot_update = load_countries(link)
        return df, country_update, song_update, plot_update

def update_songs(df, country):
        return load_songs(df, country)

def update_plot(df, country, song):
        return plot_song(df, country, song)

In [4]:
with gr.Blocks() as demo:

    gr.Markdown("## Spotify Ranking Visualizer")
    gr.Markdown("""
    This webapp allows you to visualize dynamically the Spotify rankings for many songs over multiple weeks and in many countries.

    ### 🔹 How to Use:
    1. **Select a Dataset and Enter** by inputing the kaggle link. A default dataset is already filled.
    2. **Select a newly loaded Country** from the dropdown list.
    3. **Select a newly loaded Song** from the dropdown list.
    4. A visualization is then generated for the evolution of the ranking of the selected song in the selected country.

    ### Note :
    The dataset contains ranking information from 2016/12/29 to 2022/07/14.
    """)

    df_state = gr.State(None)

    with gr.Column():
            link_input = gr.Textbox(label="Kaggle Dataset Link", value ="yelexa/spotify200")
            country_dropdown = gr.Dropdown(label="Country", choices=[], interactive=True)
            song_dropdown = gr.Dropdown(label="Song", choices=[], interactive=True)

    with gr.Column():
            plot_output = gr.Plot()

    # 1. Load Dataset -> Set Country Dropdown
    link_input.submit(
        load_dataset_and_countries,
        inputs=link_input,
        outputs=[df_state, country_dropdown, song_dropdown, plot_output],
    )

    # 2. Select Country -> Set Song Dropdown
    country_dropdown.change(
        update_songs,
        inputs=[df_state, country_dropdown],
        outputs=[song_dropdown, plot_output],
    )

    # 3. Select Song -> Display Plot
    song_dropdown.change(
        update_plot,
        inputs=[df_state, country_dropdown, song_dropdown],
        outputs=plot_output,
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0cdbeaf420da22e843.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
